<a href="https://colab.research.google.com/github/azydek0/python-homework_04/blob/arkadiusz/project_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install astropy
!pip install astroplan

In [5]:
!pip install timezonefinder[numba]
!pip install geopy

In [6]:
from astropy import constants as const
from astropy import units as u
import numpy as np
import matplotlib
%matplotlib inline
from matplotlib import pylab as plt
from astroplan import Observer
from astropy.time import Time
import pytz
from timezonefinder import TimezoneFinder
from geopy import geocoders

In [7]:
local_tz = pytz.timezone('Europe/Warsaw')
local_tz

<DstTzInfo 'Europe/Warsaw' LMT+1:24:00 STD>

In [8]:
katowice = Observer(longitude=19*u.deg + 1*u.arcmin, latitude=50*u.deg + 16 * u.arcmin, elevation=230*u.m, name="Katowice", timezone=local_tz)
katowice

<Observer: name='Katowice',
    location (lon, lat, el)=(19.016666666666666 deg, 50.266666666666666 deg, 230.00000000008052 m),
    timezone=<DstTzInfo 'Europe/Warsaw' LMT+1:24:00 STD>>

In [9]:
nowtime = Time.now()
nowtime

<Time object: scale='utc' format='datetime' value=2020-11-23 20:02:51.326950>

In [10]:
nowtime.to_datetime(timezone=local_tz)

datetime.datetime(2020, 11, 23, 21, 2, 51, 326950, tzinfo=<DstTzInfo 'Europe/Warsaw' CET+1:00:00 STD>)

In [11]:
sunrise = katowice.sun_rise_time(nowtime)

In [12]:
sunrise.ymdhms

(2020, 11, 24, 6, 18, 32.14226782)

In [13]:
result = katowice.sun_altaz(sunrise)
result

<SkyCoord (AltAz: obstime=2459177.762872017, location=(3862287.59388808, 1331149.09201147, 4881979.18528353) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt, distance) in (deg, deg, m)
    (123.48217942, 0.0029168, 1.47687082e+11)>

In [27]:
# Klasa - arg1= lokalizacja (string, format: 'Miasto, Kraj'), arg2= data (format: "RRRR-MM-DD", ale może działają też inne, trzeba sprawdzić)
class TravelerInTimeAndSapce():
    def __init__(self, location, date):
        locator = geocoders.Nominatim(user_agent='myGeocoder').geocode(location)
        date_time = Time
        timezone = TimezoneFinder().timezone_at(lng=locator.longitude , lat=locator.latitude)
        self.lat = locator.latitude * u.deg
        self.lon = locator.longitude * u.deg
        self.address = locator.address
        self.time = Time(date).to_datetime(pytz.timezone(timezone))
        self.tzname = self.time.timetz().tzname()
    def __repr__(self):
        return ('lat=' + str(self.lat) + 
            '; lon=' + str(self.lon) + 
            '; address=' + str(self.address) + 
            '; time=' + str(self.time) + 
            '; tz=' + str(self.tzname))

In [21]:
# Użycie i wywołanie metod powyższej klasy
traveler = TravelerInTimeAndSapce('Paris, France', '1999-01-01')
print(traveler.address)
print(traveler.lat, traveler.lon)
print(traveler.time)
print(traveler.tzname)
traveler

Paris, Île-de-France, France métropolitaine, 75004, France
48.8566969 deg 2.3514616 deg
1999-01-01 01:00:00+01:00
Europe/Paris


lat=48.8566969 deg; lon=2.3514616 deg; address=Paris, Île-de-France, France métropolitaine, 75004, France; time=1999-01-01 01:00:00+01:00; tz=Europe/Paris